In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Import useful libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# <font color='red'>**Loading trained networks**</font>

In [ ]:
path_model = "/home/Data/franklin/Doctorado/parkinson/projects/parcellation_translation/models/classifier/mri_spect/preprocessed/only_mri/Vgg16MRI.h5"
model = tf.keras.models.load_model(path_model, compile=False)
model.summary()

In [ ]:
l1 = model.get_layer(name='dropout_2')
emb = Model(model.input, l1.output)
emb.summary()

# Getting and saving the embeddings

In [ ]:
#loading the synthetic data
split = "test"

#synthetic_data = "../imgs_results/full_rois/preprocessed/mri_to_spect/"    
gen_path = "../../../../../../Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/"

csv_train = os.path.join(gen_path + 'control_pd_MRI_fullRois_TRAIN.csv')
csv_test = os.path.join(gen_path + 'control_pd_MRI_fullRois_TEST.csv')
train_df = pd.read_csv(csv_train, header=None)
train_df.columns = ['path', 'label']
test_df = pd.read_csv(csv_test, header=None)
test_df.columns = ['path', 'label']


print(train_df.groupby('label').count())
print(test_df.groupby('label').count())

train_df[['case_number', 'slice_number']] = train_df['path'].str.extract(r'_case_(\d+)_slice_(\d+).png').astype(int)
train_df_v2 = train_df[(train_df['slice_number'] > 41) & (train_df['slice_number'] < 132)]
train_df_v2.drop('slice_number', axis=1, inplace=True)
train_df_v2.drop('case_number', axis=1, inplace=True)

#test
test_df[['case_number', 'slice_number']] = test_df['path'].str.extract(r'_case_(\d+)_slice_(\d+).png').astype(int)
test_df_v2 = test_df[(test_df['slice_number'] > 41) & (test_df['slice_number'] < 132)]
test_df_v2.drop('slice_number', axis=1, inplace=True)
test_df_v2.drop('case_number', axis=1, inplace=True)

print("lengh of train: {}, lengh of test: {}".format(len(train_df_v2), len(test_df_v2)))

In [ ]:
train_ctrl = train_df_v2[train_df_v2['label'] == "control"]
train_pd = train_df_v2[train_df_v2['label'] == "parkinson"]

test_ctrl = test_df_v2[test_df_v2['label'] == "control"]
test_pd = test_df_v2[test_df_v2['label'] == "parkinson"]

In [ ]:
test_df_v2.iloc[0]['path']

In [ ]:
#getting embeddings for train and test

label, pred = [], []    
df = train_ctrl

for i in range(len(df)):
    img = tf.keras.preprocessing.image.load_img(df.iloc[i]['path'], target_size=(256, 256))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img/255.0
    img = tf.expand_dims(img, axis=0)
    emb_test = emb.predict(img)
    pred.extend(emb_test)
    label.append(df.iloc[i]['label'])

#pred = np.squeeze(pred, axis=1)
pred = np.array(pred)
print("dimension of predic: ", pred.shape)

#label = np.squeeze(label, axis=1)
label = np.array(label)
print("dimension of label: ", label.shape)

In [ ]:
#saving the numpy arrays
gen_path = "/home/Data/franklin/Doctorado/parkinson/projects/parcellation_translation/embeddings/classifier/preprocessed/mri/"

if df.equals(test_ctrl):
    np.save(gen_path + "test_control/controlEmbTest.npy", pred)
    np.save(gen_path + "test_control/controlLabTest.npy", label)
elif df.equals(test_pd):
    np.save(gen_path + "test_pd/parkinsonEmbTest.npy", pred)
    np.save(gen_path + "test_pd/parkinsonLabTest.npy", label)
elif df.equals(train_ctrl):
    np.save(gen_path + "train_control/controlEmbTrain.npy", pred)
    np.save(gen_path + "train_control/controlLabTrain.npy", label)
else:
    np.save(gen_path + "train_pd/parkinsonEmbTrain.npy", pred)
    np.save(gen_path + "train_pd/parkinsonLabTrain.npy", label)

In [ ]:
#haciendo dataframe
df = pd.DataFrame(list(zip(label, pred)), columns=['clase', 'predicción'])
df.groupby('clase').count()

# Dimention reduction using Tsne

In [ ]:
tsne = TSNE(n_components = 2, init = 'pca')
P1_tsne = tsne.fit_transform(pred)
print(P1_tsne.shape)

In [ ]:
l1 = P1_tsne[:,0]
l2 = P1_tsne[:,1]

df = df.drop(columns='predicción')
df['x'] = l1
df['y'] = l2

In [ ]:
conditions = [
    (df['clase'] == 'CuNi1'),
    (df['clase'] == 'CuNi2'),
    (df['clase'] == 'CuNi3')
    ]

values = [1, 2, 3]

df['labels'] = np.select(conditions, values)

In [ ]:
df

# Center of mass and distance between classes

In [ ]:
clases = ['CuNi1', 'CuNi2', 'CuNi3']
full_x_com = []
full_y_com = []
for clase in clases:
    df_clase = df[df['clase']== clase]
    #center of mass x and y axes
    x_com = df_clase['x'].sum()/len(df_clase)    
    y_com = df_clase['y'].sum()/len(df_clase) 
    full_x_com.append(x_com)
    full_y_com.append(y_com)
    
#print("====== about intra classes distances ========")
cuni1_cuni2_dis = np.sqrt(np.power(full_x_com[0]-full_x_com[1],2)+ np.power(full_y_com[0]-full_y_com[1],2))
cuni1_cuni3_dis = np.sqrt(np.power(full_x_com[0]-full_x_com[2],2)+ np.power(full_y_com[0]-full_y_com[2],2))
cuni2_cuni3_dis = np.sqrt(np.power(full_x_com[1]-full_x_com[2],2)+ np.power(full_y_com[1]-full_y_com[2],2))
#print("====== about inter classes distances ========")
x_cuni1_mean = df[df['clase']=='CuNi1']['x'].mean()
y_cuni1_mean = df[df['clase']=='CuNi1']['y'].mean()
x_cuni2_mean = df[df['clase']=='CuNi2']['x'].mean()
y_cuni2_mean = df[df['clase']=='CuNi2']['y'].mean()
x_cuni3_mean = df[df['clase']=='CuNi3']['x'].mean()
y_cuni3_mean = df[df['clase']=='CuNi3']['y'].mean()
cuni1_dis = np.sqrt(np.power(full_x_com[0]-x_cuni1_mean,2)+ np.power(full_y_com[0]-y_cuni1_mean,2))
cuni2_dis = np.sqrt(np.power(full_x_com[1]-x_cuni2_mean,2)+ np.power(full_y_com[1]-y_cuni2_mean,2))
cuni3_dis = np.sqrt(np.power(full_x_com[2]-x_cuni3_mean,2)+ np.power(full_y_com[2]-y_cuni3_mean,2))

In [ ]:
print("============ ABOUT CENTER OF MASS ==============")
print('el centro de masa para CuNi1 es: x {} and y {}'.format(full_x_com[0], full_y_com[0]))
print("el centro de masa para CuNi2 es: x {} and y {}".format(full_x_com[1], full_y_com[1]))
print("el centro de masa para CuNi3 es: x {} and y {}".format(full_x_com[2], full_y_com[2]))
print("============ ABOUT INTRA CLASS DISTANCES ==============")
print("la distancia CuNi1-CuNi2 es: {}".format(cuni1_cuni2_dis))
print("la distancia CuNi1-CuNi3 es: {}".format(cuni1_cuni3_dis))
print("la distancia CuNi2-CuNi3 es: {}".format(cuni2_cuni3_dis))
print("============ ABOUT INTER CLASS DISTANCES ==============")
print("la distancia CuNi1-CuNi1 es: {}".format(cuni1_dis))
print("la distancia CuNi2-CuNi2 es: {}".format(cuni2_dis))
print("la distancia CuNi3-CuNi3 es: {}".format(cuni3_dis))

In [ ]:
fig, ax = plt.subplots()

colors = {'CuNi1':'red', 'CuNi2':'green', 'CuNi3':'blue'}

grouped = df.groupby('clase')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key])

ax.scatter([full_x_com[0]], [full_y_com[0]], color='black', s=250)
ax.scatter([full_x_com[1]], [full_y_com[1]], color='yellow', s=250)
ax.scatter([full_x_com[2]], [full_y_com[2]], color='orange', s=250)
plt.show()